In [76]:
import requests
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

# 1. 收集和读取数据

## 下载并读取image-predictions.tsv文件

In [2]:
# 下载image-predictions.tsv文件到本地
url = 'https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/image-predictions.tsv'
r = requests.get(url)
with open('image-predictions.tsv', 'wb') as file:
    file.write(r.content)

# 读取本地下载的文件
image_predictions = pd.read_table('image-predictions.tsv', sep = '\t')
## 亦可直接读取url中文件: image_predictions = pd.read_table(url, sep = '\t')

## 读取tweet_json.txt数据
- 字符串中含有\n,无法直接读取,需要按照\n进行划分再读取
- 切分后最后一个字符串为空字符串,需要剔除

In [3]:
# 将txt文件写入tweet_json中成长字符串
with open('tweet_json.txt', 'r') as file:
    tweet_json = file.read()

# 将字符串按照\n进行切分存入列表
tweet_json_list = tweet_json.split('\n')
# 其中最后一个字符串不符合规则
tweet_json_list[-1]

''

In [ ]:
# 将上述字符串列表以json格式载入为新的字典列表
tweet_list = [json.loads(s) for s in tweet_json_list[:-1]]
# 将字典列表读入为DataFrame
tweet_df = pd.DataFrame.from_dict(json_normalize(tweet_list))

In [124]:
# 挑出含NaN值大于1000个数的列
nan_columns = list(tweet_df.isnull().sum()[tweet_df.isnull().sum() > 1000].index)

# 去除含有NaN的列
tweet_df_sub = tweet_df.drop(nan_columns, axis=1)

# 发现大部分列都没啥用,选取 tweet ID、retweet_count 和 favorite_count 字段
tweet_df_sub = tweet_df[['id_str', 'retweet_count', 'favorite_count']]

## 读取twitter-archive-enhanced.csv数据

In [5]:
twitter_archive_enhanced = pd.read_csv('twitter-archive-enhanced.csv')

# 2. 评估数据

## 目测评估

### `tweet_df_sub `表格
因为只选择了id,转发数和喜欢数,目测评估一切正常.

In [28]:
# 预览tweet_df_sub数据
tweet_df_sub.sample(5)

,id,retweet_count,favorite_count
345,831670449226514432,2053,11439
75,878057613040115712,7118,42743
0,892420643555336193,8842,39492
2282,667182792070062081,6589,15028
773,776113305656188928,5024,13050


### `image_predictions`表格

看上去也好像没啥问题.

In [125]:
# 预览image_predictions数据随机5行
image_predictions.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1192,739979191639244800,https://pbs.twimg.com/media/CkTvJTdXAAAEfbT.jpg,1,Irish_water_spaniel,0.285800,True,wig,0.240653,False,toy_poodle,0.074914,True
1519,787717603741622272,https://pbs.twimg.com/media/Cu6I9vvWIAAZG0a.jpg,3,German_shepherd,0.992339,True,malinois,0.004920,True,kelpie,0.000853,True
496,675781562965868544,https://pbs.twimg.com/media/CWDbv2yU4AARfeH.jpg,1,Maltese_dog,0.921968,True,West_Highland_white_terrier,0.017811,True,toy_poodle,0.013555,True
1901,851591660324737024,https://pbs.twimg.com/media/C9F2FG5WAAAJ0iN.jpg,1,Cardigan,0.394507,True,Chihuahua,0.077254,True,French_bulldog,0.076559,True
1720,819347104292290561,https://pbs.twimg.com/media/C17n1nrWQAIErU3.jpg,3,Rottweiler,0.909106,True,black-and-tan_coonhound,0.044120,True,Doberman,0.031835,True


### `twitter_archive_enhanced`表格

#### 整洁问题
- `source`包含了整个标签`<a>`的信息;
- `timestamp`包含年月日和具体时间;
- 包含很多NaN的列,如`retweeted_status_id`等;

In [9]:
# 预览twitter_archive_enhanced数据
twitter_archive_enhanced.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
587,799757965289017345,NaN,NaN,2016-11-18 23:35:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Sobe. She's a h*ckin happy doggo. Only...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/799757965...,13,10,Sobe,doggo,None,None,None
364,829141528400556032,NaN,NaN,2017-02-08 01:35:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Malcolm. He goes from sneaky tongue sl...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/829141528...,12,10,Malcolm,None,None,None,None
499,813127251579564032,NaN,NaN,2016-12-25 21:00:18 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's an anonymous doggo that appears to be v...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/813127251...,11,10,None,doggo,None,None,None
1022,746542875601690625,NaN,NaN,2016-06-25 03:17:46 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",Here's a golden floofer helping with the groce...,NaN,NaN,NaN,https://vine.co/v/5uZYwqmuDeT,11,10,None,None,floofer,None,None
2264,667538891197542400,NaN,NaN,2015-11-20 03:04:08 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",This is a southwest Coriander named Klint. Hat...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667538891...,9,10,a,None,None,None,None


## 编程评估

### `tweet_df_sub`表格

#### 质量问题
- 转推数和喜欢数最小值为0

In [127]:
tweet_df_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 3 columns):
id_str            2352 non-null object
retweet_count     2352 non-null int64
favorite_count    2352 non-null int64
dtypes: int64(2), object(1)
memory usage: 55.2+ KB


In [140]:
tweet_df_sub.describe()

,retweet_count,favorite_count
count,2352.000000,2352.000000
mean,3134.932398,8109.198980
std,5237.846296,11980.795669
min,0.000000,0.000000
25%,618.000000,1417.000000
50%,1456.500000,3596.500000
75%,3628.750000,10118.000000
max,79116.000000,132318.000000


### `image_predictions`表格

#### 整洁问题
- `tweet_id`为数值型,要转化为字符串;

#### 质量问题
- `img_num`值有异常值;

In [141]:
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [142]:
image_predictions.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


### `twitter_archive_enhanced`表格

#### 整洁问题
- `tweet_id`为数值型;
- 狗的地位几列分为了好几列;

#### 质量问题
- `in_reply_to_status_id`、`in_reply_to_user_id`、`retweeted_status_id`等列存在大量缺失值;
- `name`列有很多无效值,如a,an,the等;
- `rating_numerator`和`rating_denominator`两列存在异常值;

In [143]:
twitter_archive_enhanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [146]:
twitter_archive_enhanced.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1517,690938899477221376,NaN,NaN,2016-01-23 16:47:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...","She thought the sunset was pretty, but I thoug...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/690938899...,10,10,None,None,None,None,None
1626,684800227459624960,NaN,NaN,2016-01-06 18:14:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Theodore. He's dapper as hell. Probably o...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/684800227...,11,10,Theodore,None,None,None,None
2280,667393430834667520,NaN,NaN,2015-11-19 17:26:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Fwed. He is a Canadian Asian Taylormad...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667393430...,8,10,Fwed,None,None,None,None
419,822244816520155136,NaN,NaN,2017-01-20 00:50:15 +0000,"<a href=""http://twitter.com/download/iphone"" r...",We only rate dogs. Please don't send pics of m...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/822244816...,11,10,None,None,None,None,None
2209,668623201287675904,NaN,NaN,2015-11-23 02:52:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Jomathan. He is not thrilled about the...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/668623201...,10,10,Jomathan,None,None,None,None


In [144]:
twitter_archive_enhanced.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [147]:
twitter_archive_enhanced.name.value_counts()

None         745
a             55
Charlie       12
Lucy          11
Cooper        11
Oliver        11
Tucker        10
Lola          10
Penny         10
Bo             9
Winston        9
Sadie          8
the            8
Bailey         7
an             7
Toby           7
Daisy          7
Buddy          7
Leo            6
Jax            6
Scout          6
Oscar          6
Jack           6
Bella          6
Rusty          6
Stanley        6
Dave           6
Milo           6
Koda           6
George         5
            ... 
Julio          1
Kona           1
Ginger         1
Ike            1
Cedrick        1
Henry          1
Jeremy         1
Zoe            1
Tanner         1
Emanuel        1
Rufio          1
Simba          1
Harvey         1
Tupawc         1
Alejandro      1
Bluebert       1
Jazz           1
Bobble         1
Mimosa         1
Emmie          1
Dug            1
Batdog         1
Trevith        1
Bert           1
Loomis         1
O              1
Laela          1
William       

## 综合评估三个数据
- 三个数据集都有tweet id变量,其中`tweet_df_sub`中为`id`,其他两个数据集为`tweet_id`;

In [158]:
# 三个数据集中所有的列
all_columns = pd.Series(list(tweet_df_sub) + list(image_predictions) + list(twitter_archive_enhanced))
# 三个数据集中相同的列
print('相同的列是%s\n'%list(all_columns[all_columns.duplicated()]))
print('所有的列有%s'%list(all_columns))

相同的列是['tweet_id']

所有的列有['id_str', 'retweet_count', 'favorite_count', 'tweet_id', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog', 'tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp', 'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator', 'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo']


# 3. 清洗数据

## `twitter_archive_enhanced`表格

#### 整洁问题
- `source`包含了整个标签`<a>`的信息;
- `timestamp`包含年月日和具体时间;
- 包含很多NaN的列,如`retweeted_status_id`等;